# Inference Notebook — Exam Reference Sheet Generator

**Best to run it here on Google Colab: https://colab.research.google.com/drive/1JcPd02BrtB2o1V6OIxhLVjCzQASN2KBf#scrollTo=0QKHIGvjA6IR**

This notebook loads the fine-tuned model and generates exam reference sheets from:

- A notes PDF
- An exam topics PDF

It calls a single end-to-end pipeline that:
- Extracts text and diagrams
- Generates a compressed cheat sheet
- Renders a page-limited PDF with selected diagrams

In [1]:
!pip install transformers datasets accelerate sentencepiece
!pip install pymupdf reportlab matplotlib pandas

In [2]:
REPO_URL = "https://github.com/arely83/CS372FinalProject.git"
REPO_NAME = "CS372FinalProject"

import os

if not os.path.exists(REPO_NAME):
    !git clone {REPO_URL}
%cd {REPO_NAME}
print("CWD is now:", os.getcwd())
!ls

/content/CS372FinalProject
CWD is now: /content/CS372FinalProject
 data			        exam_topics_game.pdf	 'notes_ecs (1).pdf'
 docs			        exam_topics_gd.pdf	  notes_ecs.pdf
 environment.yml	        generated_reference.pdf   notes_game.pdf
'exam_topics_current (1).pdf'   models			  notes_gd.pdf
'exam_topics_current (2).pdf'   notebooks		  README.md
 exam_topics_current.pdf       'notes_current (1).pdf'	  runtime_imgs
'exam_topics_ecs (1).pdf'      'notes_current (2).pdf'	  src
 exam_topics_ecs.pdf	        notes_current.pdf


##Retrieving saved model from Google Drive



In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

!mkdir -p models
!cp -r "/content/gdrive/MyDrive/CS372FinalProject/models/cheatsheet_model" models/cheatsheet_model

import os
print("Local model dir exists?", os.path.isdir("models/cheatsheet_model"))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Local model dir exists? True


## Upload Exam PDFs

Upload two PDFs from your local machine:
- Your class notes
- The exam topic list

These will be moved into `data/samples/` for the pipeline to use.

In [4]:
from google.colab import files
uploaded = files.upload()   # choose your notes + topics PDFs

Saving exam_topics_ecs.pdf to exam_topics_ecs (2).pdf
Saving notes_ecs.pdf to notes_ecs (2).pdf


# Run the Reference Sheet Generation Pipeline

This will:
- Parse `NOTES_PDF` and `TOPICS_PDF`
- Generate a compact cheat sheet constrained to a fixed number of pages
- Select relevant diagrams from the notes
- Render the final PDF to `data/runtime/gen_reference.pdf`

Modify the cell below with relevant information:

In [5]:
NOTES_PDF = "notes_ecs.pdf"
TOPICS_PDF = "exam_topics_ecs.pdf"
NUM_PAGES = 2

In [6]:
from src.pipeline import generate_reference_sheet_pipeline

sheet_text, image_paths, out_pdf = generate_reference_sheet_pipeline(
    notes_pdf_path=NOTES_PDF,
    exam_topics_pdf_path=TOPICS_PDF,
    num_pages=NUM_PAGES,
    img_dir="runtime_imgs",
    model_dir="models/cheatsheet_model",
    out_pdf="gen_reference.pdf",
)

print("Generated PDF:", out_pdf)

Generated PDF: gen_reference.pdf


## Preview Generated Cheat Sheet Text

You can inspect the first part of the generated cheat sheet below.

In [7]:
print(sheet_text[:2000])

striations • Earth's magnetic field: • Paleomagnetism #1: Apparent Polar Wander • Paleomagnetism #2: Magnetic Reversals • Plate tectonics • Divergent Plate Boundaries • Convergent Plate Boundaries • Convergent Plate Boundaries (subduction vs collision zones) • Strike-slip (transform) plate Boundaries • Mantle Plume (hotspot) tracks • Plate velocities • What drives plate tectonics? • Mantle Convection • Tectonic (lithospheric) processes: Slab Pull, Ridge Push • Rising or descending limbs of convection cells (relation to hotspots & subduction zones) • Mineral identification • Mineral classification (silicate vs non-silicate) • Silicate Minerals • Silica Tetrahedron • Five common silicate structures (& six silicate minerals to know- see slides for names, silicate structures, & composition as indicated in red) • Felsic vs mafic minerals • Silica Tetrahedron • Six major igneous rocks • How magma is produced in the mantle by melting peridotite • Phase diagram • Geothermal Gradient • Decompre

## Download Generated Cheat Sheet PDF

Use the cell below to download the final PDF to your local machine.

In [8]:
from google.colab import files
files.download("gen_reference.pdf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>